In [1]:
# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import time
import os
import math

import numpy as np
from numpy.random import seed
from numpy.random import rand

from engine import *

# seed random number generator
seed(1)

pybullet build time: Nov 28 2023 23:51:11


In [2]:
# create a pybullet engine from the engine module
engine = PyBulletEnvironment()


In [3]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robot_urdf = './urdf/shovel/shovelFlat.urdf'

# load the robot
engine.open_environment(robot_urdf, startPos, startOrientation)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org


MESA: warning: Driver does not support the 0xa7a0 PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: iris
MESA: warning: Driver does not support the 0xa7a0 PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: iris


In [4]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(engine.ID[1], 0)

# admissible init positions
minpos = np.asarray(shovelLinkState[0]) - 3*1e-1
maxpos = np.asarray(shovelLinkState[0]) + 3*1e-1

# number of pebbles
pebbleNum = 50

# store pebbles IDs
pebbleId = np.zeros(pebbleNum)

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    engine.load_urdf('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)    
    
# simulate
engine.simulate(2)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inerti

In [5]:
# test move with pos

# max force on joint
maxForce = 1000

# target pos and vel - prismatic
targetPosMean = 0
targetPosStd = 3e-1
targetVelPos = 0.2

# target pos and vel - revolute
targetAngMean = 0
targetAngStd = 2.5e-1
targetVelAng = 1

# current control mode
mode = p.VELOCITY_CONTROL

# init target Pos and Vel    
targetPos= targetPosMean+targetPosStd
targetAng= targetAngMean+targetAngStd

# control
for i in range(5):        
    
    # error init - Pos
    shovelJointPos = p.getJointState(engine.ID[1],0)
    ePos = targetPos-shovelJointPos[0]
    
    # error init - Ang
    shovelJointAng = p.getJointState(engine.ID[1],1)
    eAng = targetAng-shovelJointAng[0]
    
    # control
    while np.abs(ePos) > 1e-2:
        
        # control - Pos
        if np.abs(ePos) > 1e-2:
            targetVelPosCtrl = targetVelPos*np.sign(ePos)
            shovelContr = p.setJointMotorControl2(bodyIndex=engine.ID[1], jointIndex=0, controlMode=mode, targetVelocity=targetVelPosCtrl, force=maxForce)        
        
        # control - Ang
        if np.abs(eAng) > 1e-2:
            targetVelAngCtrl = targetVelAng*np.sign(eAng)
            shovelContr = p.setJointMotorControl2(bodyIndex=engine.ID[1], jointIndex=1, controlMode=mode, targetVelocity=targetVelAngCtrl, force=maxForce)         
        
        # info on target
        # print(str(targetVelPosCtrl), str(targetVelAngCtrl))
        # print(str(shovelJointPos[0]), str(shovelJointAng[0]))
        # print(str(ePos), str(eAng))
        
        # step
        p.stepSimulation()
        
        # sleep
        time.sleep(1/240)
        
        # error update - Pos
        shovelJointPos = p.getJointState(engine.ID[1],0)
        ePos = targetPos-shovelJointPos[0] 
        
        # error update - Ang
        shovelJointAng = p.getJointState(engine.ID[1],1)
        eAng = targetAng-shovelJointAng[0]   
        
    # redefine targets
    targetPos= targetPosMean + (targetPosMean-targetPos)
    targetAng= targetAngMean + (targetAngMean-targetAng)

In [6]:
# disconnect engine
engine.close_environment()

numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
